# Práctica 1

## Preparación del entorno

In [70]:
import numpy as np
import keras as ker
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import PIL
import os

from PIL import Image
from IPython.display import clear_output # Limpiar output
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from numpy import asarray

# SETUP

## Procesamiento de datos

In [71]:
"""
    Primer paso es convertir las imagenes en un array y etiquetarlas.
    Primero de todo, en el siguiente bucle recorreremos todos los ficheros de entrenamiento, convirtiendo cada imagen en una matriz (normalizada / 255)
    Una vez la tenemos normalizada, la añadimos al conjunto de entrenamiento
    Como cada imagen esta ya en una carpeta según su clase, guardamos de que carpeta pertenece la imagen y así la etiquetamos en las salidas esperadas
"""
entradas_entrenamiento = []
salidas_entrenamiento = []
with os.scandir('../Dataset/imgs/train') as ficheros: # Iteramos todo el conjunto de entrenamiento
    for fichero in ficheros:
        path = os.path.join(fichero)
        tipo_clase = path.split("/")
        clase_img = tipo_clase[4]
        print("Leyendo imágenes de la carpeta ",clase_img,end="\r")
        with os.scandir(path) as imagenes:
            for imagen in imagenes:
                img_ruta = os.path.join(imagen)
                img = Image.open(img_ruta)
                img_pixeles = asarray(img) 
                img_normalizada = img_pixeles / 255 # Normalizamos
                entradas_entrenamiento.append(img_normalizada)
                salidas_entrenamiento.append(clase_img)
print("Todas las imágenes del entrenamiento leidas y guardadas")